# Query & lookup data

Querying is based on [SQLAlchemy](https://github.com/sqlalchemy/sqlalchemy) `select` statements.

```{toctree}
:hidden:

query-book
```

In [ ]:
import lamindb as ln

ln.track()

## Basic select statements

Let's get a lookup object to have autocomplete for query targets.

In [ ]:
user_handles = ln.User.lookup(field="handle")

Query a user record:

In [ ]:
user = ln.select(ln.User, handle=user_handles.testuser1).one()

Query all files created by that user:

In [ ]:
stmt = ln.select(ln.File, created_by_id=user.id)

To access the query results encoded in `stmt` (a {class}`~lamindb.dev.db.SelectStmt`), execute it with one of

- `.all()`: A list of records.
- `.df()`: A dataframe with each record stored as a row.
- `.one()`: Exactly one record. Will raise an error if there is none.
- `.one_or_none()`: Either one record or `None` if there is no query result.

For example:

In [ ]:
stmt.all()[:3]

It's often the most convenient to use the built-in converter to DataFrames.

In [ ]:
stmt.df().head()

## Joined views

Say we want all user information in this table.

In [ ]:
ln.select(ln.User, ln.File).join(ln.File.created_by).df().head()

Say, we only want the user handle.

In [ ]:
ln.select(ln.User.handle, ln.File).join(ln.File.created_by).df().head()

Say, we only want selected information from all tables.

In [ ]:
ln.select(ln.File.name, ln.File.suffix, ln.File.size, ln.User.handle).join(
    ln.File.created_by
).df().head()

## More filtering

Let us subset to just the parquet files - we know it's exactly a single one. So we can get the record using `.one()`.

In [ ]:
ln.select(ln.File, suffix=".parquet", created_by_id=user.id).df()

Or subset to files greater than 10kB. Here, we can't use keyword arguments, but need an explicit where statement.

In [ ]:
stmt = (
    ln.select(ln.File)
    .where(ln.File.size > 1e4)
    .join(ln.File.created_by)
    .where(ln.User.handle == user_handles.testuser1)
)
stmt.df()

Or select a notebook based on a substring in the name:

In [ ]:
ln.select(ln.Transform, type="notebook").where(
    ln.Transform.title.contains("Track")
).df()

## Load and stage

Load data objects into the work environment via {meth}`~lamindb.File.load`:

In [ ]:
file = ln.select(ln.File, name="iris").first()

df = file.load()

If there is a canonical in-memory representation (like a dataframe), data is loaded directly into memory.

In [ ]:
df.head()

If you want a local filepath, use stage:

In [ ]:
file = ln.select(ln.File).where(ln.File.name.contains("paradisi05")).one()

In [ ]:
file.stage()